In [ ]:
#!pip install -q torch==1.7.1+cpu torchvision==0.8.2+cpu torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
import torch
a = torch.tensor([[1,2,3,4,5],[6,7,8,9,10],[11,12,13,14,15]])
b = torch.tensor([[6,7,8,9,10],[11,12,13,14,15],[1,2,3,4,5]])
torch.cat((a,b),dim = 1)

In [ ]:
#!pip install -q --no-index torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cpu.html
#!pip install -q --no-index torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cpu.html
#!pip install -q --no-index torch-cluster -f https://pytorch-geometric.com/whl/torch-1.7.0+cpu.html
#!pip install -q --no-index torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.7.0+cpu.html
#!pip install torch-geometric

# link prediction


In [1]:
import torch
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from sklearn.neural_network import MLPClassifier 
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import GradientBoostingClassifier
from torch_geometric.data import NeighborSampler
from torch.optim import lr_scheduler
import torch.optim as optim
import matplotlib.pyplot as plt
import collections
from concurrent.futures import ThreadPoolExecutor
import numpy as np
from modules.model import Net
from modules.sampling import Sampler
import random
from modules.negativeSampling import NegativeSampler


In [77]:
a = torch.randint(10,(8,10)).to('cuda')
b = torch.randint(10,(8,10)).to('cuda')

In [27]:
prob = torch.rand(a.shape)
prob=prob.to('cuda')
prob

tensor([[0.3326, 0.5794, 0.4753, 0.4698, 0.6283, 0.2083, 0.2654, 0.8725, 0.6891,
         0.4477],
        [0.4337, 0.6463, 0.4055, 0.5853, 0.0628, 0.0786, 0.8766, 0.6512, 0.5506,
         0.8531],
        [0.5603, 0.1819, 0.3964, 0.0639, 0.3416, 0.8900, 0.4981, 0.4739, 0.6932,
         0.8974],
        [0.6672, 0.0271, 0.3291, 0.9649, 0.3474, 0.8942, 0.9534, 0.6314, 0.1395,
         0.6212],
        [0.7194, 0.3476, 0.7613, 0.7796, 0.8874, 0.3902, 0.8320, 0.9551, 0.5834,
         0.2160],
        [0.2752, 0.7583, 0.6043, 0.3503, 0.0339, 0.7028, 0.4337, 0.8955, 0.7161,
         0.1990],
        [0.8454, 0.9602, 0.3244, 0.0704, 0.7366, 0.1887, 0.2671, 0.6973, 0.8424,
         0.3890],
        [0.8389, 0.4397, 0.6772, 0.6053, 0.5383, 0.3185, 0.5720, 0.9564, 0.2286,
         0.8860]], device='cuda:0')

In [49]:
2/3

tensor([[1., 1., 1., 1., 1., 1., 1., 0., 1., 1.],
        [1., 1., 1., 1., 1., 1., 0., 1., 1., 0.],
        [1., 1., 1., 1., 1., 0., 1., 1., 1., 0.],
        [1., 1., 1., 0., 1., 0., 0., 1., 1., 1.],
        [0., 1., 0., 0., 0., 1., 0., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 0., 0., 1.],
        [0., 0., 1., 1., 0., 1., 1., 1., 0., 1.],
        [0., 1., 1., 1., 1., 1., 1., 0., 1., 0.]], device='cuda:0')

In [ ]:
3,3,,6, ,3,0

In [78]:
print(a[:,1:])
print(b[:,1:])
ten= (a[:,1:] == b[:,1:])
indices = (ten == True).nonzero().t()[1]
import collections
indices


tensor([[5, 0, 5, 7, 0, 2, 7, 5, 2],
        [2, 6, 4, 7, 5, 7, 8, 0, 6],
        [1, 8, 7, 8, 4, 3, 3, 7, 8],
        [1, 9, 4, 2, 3, 2, 5, 3, 1],
        [3, 2, 9, 0, 4, 2, 6, 2, 8],
        [7, 2, 3, 3, 4, 5, 9, 8, 6],
        [0, 1, 7, 7, 1, 3, 9, 7, 6],
        [1, 5, 6, 6, 0, 0, 7, 7, 0]], device='cuda:0')
tensor([[4, 2, 4, 5, 8, 3, 4, 1, 9],
        [1, 1, 9, 4, 1, 2, 5, 4, 1],
        [3, 0, 5, 3, 0, 7, 1, 7, 9],
        [3, 7, 2, 7, 1, 1, 6, 1, 2],
        [8, 4, 1, 5, 5, 9, 0, 9, 4],
        [1, 8, 5, 3, 9, 6, 6, 8, 3],
        [4, 2, 5, 7, 1, 2, 3, 1, 9],
        [1, 9, 2, 9, 2, 3, 2, 2, 4]], device='cuda:0')


tensor([7, 3, 7, 3, 4, 0], device='cuda:0')

In [ ]:
torch.tensor(a==b)

In [92]:
torch.tensor(a!=7)

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


tensor([[ True,  True,  True,  True, False,  True,  True, False,  True,  True],
        [ True,  True,  True,  True, False,  True, False,  True,  True,  True],
        [ True,  True,  True, False,  True,  True,  True,  True, False,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True, False,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True, False, False,  True,  True,  True, False,  True],
        [ True,  True,  True,  True,  True,  True,  True, False, False,  True]],
       device='cuda:0')

In [99]:
9-len(torch.unique((a==b).nonzero().t()[0]))

5

In [111]:
a1 = a!=7
a2 = a==b
a3 = b!=7
len(torch.unique((a1*a2*a3).nonzero().t()[0]))/len(a)

In [105]:
c*d

tensor([[False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False,  True, False, False, False,  True, False],
        [ True, False, False, False, False,  True, False, False, False, False],
        [False,  True, False, False, False, False, False, False, False, False]],
       device='cuda:0')

In [122]:
torch.rand(4)

tensor([0.0932, 0.7630, 0.4104, 0.0950])

In [126]:
b=torch.rand(4,4)

In [179]:
b

tensor([[0.1381, 0.8874, 0.0345, 0.3131],
        [0.8470, 0.2617, 0.7583, 0.6486],
        [0.6139, 0.4134, 0.0813, 0.4948],
        [0.4997, 0.3430, 0.6893, 0.7920]])

In [195]:
b[ind1.t()[0]]

tensor([[0.1381, 0.8874, 0.0345, 0.3131],
        [0.8470, 0.2617, 0.7583, 0.6486],
        [0.8470, 0.2617, 0.7583, 0.6486],
        [0.4997, 0.3430, 0.6893, 0.7920]])

In [184]:
ind1 = torch.tensor([1, 0, 2, 3])

In [190]:
ind1=(b>c).nonzero()
ind1.t()[0]

tensor([0, 1, 1, 3])

In [171]:
first_nonzero = torch.nonzero((b>c),as_tuple=True)
first_nonzero[0]
b[first_nonzero[0]][first_nonzero[1]]

tensor([[0.8470, 0.2617, 0.7583, 0.6486],
        [0.1381, 0.8874, 0.0345, 0.3131],
        [0.8470, 0.2617, 0.7583, 0.6486],
        [0.4997, 0.3430, 0.6893, 0.7920]])

In [166]:
first_nonzero

tensor([1, 2, 0, 1])

In [140]:
ind_new=torch.tensor([1,2,3,4])

In [148]:
c=0.7


tensor([[1., 0., 0., 0.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [ ]:
dataset =Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())
data = dataset[0]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = data.x.to(device)
y = data.y.squeeze().to(device)

In [ ]:
from torch_sparse import SparseTensor
E = data.num_edges
N = data.num_nodes
adj = SparseTensor(
            row=data.edge_index[0], col=data.edge_index[1],
            value=torch.arange(E, device=data.edge_index.device),
            sparse_sizes=(N, N))
walk_length = 2
batch_size=3
start = torch.randint(0, N, (batch_size, ), dtype=torch.long)
node_idx = adj.random_walk(start.flatten(), walk_length)
print(start)
print(node_idx)

In [ ]:
train_mask = torch.tensor([True]*int(0.8*len(data.x)+1) + [False]*int(0.2*len(data.x)))
#val_mask = torch.tensor([False]*int(0.6*len(data.x)+1) + [True]*int(0.2*len(data.x)+1)+[False]*int(0.2*len(data.x)))
test_mask = torch.tensor([False]*int(0.8*len(data.x)+1) + [True]*int(0.2*len(data.x)))

In [ ]:
#splitting data to train and test
train_edge_index = []
test_edge_index = []
indices_to_delete  = random.choices(list(range(len(data.edge_index[0]))), k = int(len(data.edge_index[0])*0.3))
for i,x in enumerate(list(zip(*data.edge_index))):
    if i in indices_to_delete:
        test_edge_index.append(x)
    else:
        train_edge_index.append(x)
test_edge_index = torch.tensor(np.array(list(zip(*test_edge_index))))
train_edge_index = torch.tensor(np.array(list(zip(*train_edge_index))))
#data.edge_index = train_edge_index 

In [ ]:
class NegativeSampler_LP(NegativeSampler):
     def negative_sampling(self, batch, num_negative_samples):
        a = self.data.edge_index
        f = self.adj_list(a)
        g = dict()
        l = batch.tolist()
        for e in l:
            if e not in g:
                g[e] = []
                g[e].append(random.choice(f[e]))
            else: 
                g[e].append(random.choice(f[e]))

        return self.torch_list(g)
num_neg_pairs = len(test_edge_index[0])
NS = NegativeSampler(data)
NS_LP= NegativeSampler_LP(data)
if num_neg_pairs>len(data.x):
    indices_start_neg_pairs = list(range(len(data.x)))
    l = random.choices(list(range(len(data.x))), k = num_neg_pairs-len(data.x))
    indices_start_neg_pairs+=l
else:
    indices_start_neg_pairs = random.choices(list(range(len(data.x))), k = num_neg_pairs-len(data.x) )
nei = NS_LP.negative_sampling(torch.tensor(indices_start_neg_pairs),1)
nei = torch.transpose(nei, 0,1)

In [ ]:
#append negative samples to test set
y_true = [1]*len(test_edge_index[0])
test_edge_index=torch.cat((test_edge_index,nei),1)
y_true+=[0]*len(nei[0])

In [ ]:
data.edge_index = train_edge_index

**Для начала посмотрим, что получится, если передать только фичи. Без обучения**

**Теперь сравним с результатом после обучения эмбедингов**

In [ ]:
SAGE = {"Name":"SAGE" , "walk length":5,"walks per node":50,"num negative samples":20,"context size" : 10,"p":1,"q":1, "loss var": "Random Walks"}
DeepWalk = {"Name": "DeepWalk","walk length":10,"walks per node":10,"num negative samples":10,"context size" : 10,"p":1,"q":1,"loss var": "Random Walks" } #Проблемы с памятью после того, как увеличила количество тренировочных данных
#DeepWalk = {"Name": "DeepWalk","walk length":40,"walks per node":80,"num negative samples":20,"context size" : 10,"p":1,"q":1,"loss var": "Random Walks" } #Проблемы с памятью после того, как увеличила количество тренировочных данных
Node2Vec = {"Name": "Node2Vec","walk length":20,"walks per node":10,"num negative samples":1,"context size" : 10,"p":1.414 ,"q":1.414, "loss var": "Random Walks"}#то же самое 
#Node2Vec = {"Name": "Node2Vec","walk length":100,"walks per node":18,"num negative samples":20,"context size" : 16,"p":1.414 ,"q":1.414, "loss var": "Random Walks"}#то же самое 

LINE2 = {"Name": "LINE2","C": "Adj","num negative samples":10,"loss var": "Context Matrix"} #настроить параметры в принципе при 0.003 что то выходит  
HOPE_RPR = {"Name": "HOPE: RPR","C":"RPR","loss var": "Factorization"}#оч большой loss


GraphFactorization = {"Name": "Graph Factorization","C":"Adj","loss var": "Factorization"} #jоч большой loss
HOPE_Katz = {"Name": "HOPE: Katz","C":"Katz","loss var": "Factorization"}#Проблемы оч большой loss
VERSE_PPR =  {"Name": "VERSE","C": "PPR","num negative samples":10,"loss var": "Context Matrix"}  #проблеиы


In [ ]:
from sklearn.metrics import precision_score, roc_auc_score

def train_lp(model,data,optimizer,Sampler,train_loader,dropout):
    model.train()        
    total_loss = 0
    optimizer.zero_grad()
    if model.mode == 'unsupervised':
        if model.conv=='GCN':
            out = model.inference(data.to(device),dp=dropout)
            samples = Sampler.sample(torch.tensor(list(range(len(data.x)))))
            loss = model.loss(out, samples)
            total_loss+=loss
        else:
            for batch_size, n_id, adjs in train_loader:
                # adjs holds a list of (edge_index, e_id, size) tuples.
                adjs = [adj.to(device) for adj in adjs]
                out = model.forward(data.x[n_id].to(device), adjs)
                samples = Sampler.sample((n_id.numpy().tolist())[:batch_size])
                loss = model.loss(out, samples)#pos_batch.to(device), neg_batch.to(device))
                #print(out.shape, samples[0].shape,samples[1].shape)
                total_loss+=loss
        total_loss.backward()
        optimizer.step()      
        return total_loss /len(train_loader)
    elif model.mode== 'supervised':
        if model.conv=='GCN':
            out = model.inference(data.to(device),dp=dropout)
            loss = model.loss_sup(out[train_mask],y[train_mask])
            total_loss+=loss 
        else:
            for batch_size, n_id, adjs in train_loader:
                adjs = [adj.to(device) for adj in adjs]
                out = model.forward(data.x[n_id].to(device), adjs)
                loss = model.loss_sup(out,y[n_id[:batch_size]])
                total_loss+=loss
        total_loss.backward(retain_graph=True)
        optimizer.step()      
        return total_loss /len(train_loader)       

@torch.no_grad()
def test_lp(model,data,classifier,y_true): 
    model.eval()
    out = model.inference(data.to(device))
    y_true = np.array(y_true)
    if model.mode == 'supervised':
        y_true = y.cpu().unsqueeze(-1)
        y_pred = out.cpu().argmax(dim=-1, keepdim=True)
        accs = []
        
        accs=[int(y_pred.eq(y_true[mas]).sum()) / int(mask.sum())]
        return accs
    
    elif model.mode == 'unsupervised':
        y_pred = []
        for x in list(zip(*test_edge_index)):
            if torch.sigmoid(torch.dot(out[x[0]],out[x[1]])):
                y_pred.append( 1 )
            else:
                y_pred.append(0)
            print(torch.sigmoid(torch.dot(out[x[0]],out[x[1]])))

        return roc_auc_score(y_true,np.array(y_pred)) 
        #return [precision_score(y_true[train_mask.cpu()], best_preds_train, average='macro'), precision_score(y_true[val_mask.cpu()], best_preds_val, average='macro'),precision_score(y_true[test_mask.cpu()], best_preds, average='macro')]

In [ ]:
#models = [Net(dataset = dataset,mode='supervised',conv='GCN',device=device), Net(dataset = dataset,mode='supervised',conv='GAT',device=device),Net(dataset = dataset,mode='supervised',conv='SGC',device=device), Net(dataset = dataset,mode='unsupervised',conv='GCN',device=device),Net(dataset = dataset,mode='unsupervised',conv='GAT',device=device),Net(dataset = dataset,mode='unsupervised',conv='SAGE',device=device)]
# lr=0.005
SAGE = {"Name":"SAGE" , "walk length":10,"walks per node":1,"num negative samples":10,"context size" :5 ,"p":1,"q":1, "loss var": "Random Walks"}
DeepWalk = {"Name": "DeepWalk","walk length":10,"walks per node":10,"num negative samples":10,"context size" : 10,"p":1,"q":1,"loss var": "Random Walks" } #Проблемы с памятью после того, как увеличила количество тренировочных данных
#DeepWalk = {"Name": "DeepWalk","walk length":40,"walks per node":80,"num negative samples":20,"context size" : 10,"p":1,"q":1,"loss var": "Random Walks" } #Проблемы с памятью после того, как увеличила количество тренировочных данных
Node2Vec = {"Name": "Node2Vec","walk length":20,"walks per node":10,"num negative samples":1,"context size" : 10,"p":1.414 ,"q":1.414, "loss var": "Random Walks"}#то же самое 
#Node2Vec = {"Name": "Node2Vec","walk length":100,"walks per node":18,"num negative samples":20,"context size" : 16,"p":1.414 ,"q":1.414, "loss var": "Random Walks"}#то же самое 

LINE2 = {"Name": "LINE2","C": "Adj","num negative samples":10,"loss var": "Context Matrix"} #настроить параметры в принципе при 0.003 что то выходит  
HOPE_RPR = {"Name": "HOPE: RPR","C":"RPR","loss var": "Factorization"}#оч большой loss


GraphFactorization = {"Name": "Graph Factorization","C":"Adj","loss var": "Factorization"} #jоч большой loss
HOPE_Katz = {"Name": "HOPE: Katz","C":"Katz","loss var": "Factorization"}#Проблемы оч большой loss
VERSE_PPR =  {"Name": "VERSE","C": "PPR","num negative samples":10,"loss var": "Context Matrix"}  #проблеиы

sizes = [[5],[10],[25],[5,10],[10,25],[5,10,25]]

hidden_layer = 64
out_layer = 128
dropout = 0.5
size = [10,25]
learning_rate = 0.003

classifier = "logistic regression"

from datetime import datetime
d = datetime.now()
for Conv in ['GCN']:
    for loss in [SAGE]:
        #train_loader = NeighborSampler(data.edge_index, batch_size = 2708, sizes=size)
       
        #LossSampler = Sampler(data,device=device,mask=train_mask,loss_info=loss)
        #model = Net(dataset = dataset,mode='unsupervised',conv=Conv,loss_function=loss["loss var"],device=device,hidden_layer=hidden_layer,out_layer =out_layer,num_layers = len(size),dropout = dropout)
        #model.to(device)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay = 1e-5)
        #scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, threshold=0.01, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
       # scheduler=lr_scheduler.StepLR(optimizer, step_size=25,gamma=0.1)
        losses=[]
        train_accs=[]
        test_accs=[]
        val_accs=[]
        name_of_plot='conv: '+model.conv+', mode: '+model.mode+', loss from '+loss["Name"]
       
        print(name_of_plot)
        
        for epoch in range(100):
            print(epoch)
            loss = train_lp(model,data,optimizer,LossSampler,train_loader,0)
            losses.append(loss)
            #print(loss)
            
            #train_accs.append(train_acc)
#test_accs.append(test_acc)
        test_acc = test_lp(model,data,'logistic regression',y_true)
           # val_accs.append(val_acc)
           # log = 'Loss: {:.4f}, Epoch: {:03d}, Train: {:.4f}, Test: {:.4f}'
            #scheduler.step()
            #print(log.format(loss, epoch, train_acc, test_acc))
        print('Test acc on the last epoch ', test_acc)
        plt.plot(losses)
        plt.title(name_of_plot+' loss')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.show()

In [ ]:
test_lp(model,data,'logistic regression',y_true)

In [ ]:
test_acc = test_lp(model,data,'logistic regression',y_true)
print('Test acc on the last epoch ', test_acc)

# optuna

In [ ]:
import optuna
from torch_geometric.datasets import Planetoid
dataset =Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())
data = dataset[0]

def objective(trial):
    # Integer parameter
    hidden_layer = 32#trial.suggest_categorical("hidden_layer", [32,64,128,256])
    out_layer = trial.suggest_int("out_layer", 32,128,step = 32,log = False)
    # Floating point parameter
    dropout = trial.suggest_float("dropout", 0.0,0.5,step = 0.1)
   # size_str = trial.suggest_categorical("size", ["[10]","[25]","[5,10]","[10,25]","[5,10,25]"])
   # b = size_str.split("[")[1].split("]")[0].split(',')
    size = [5]#list(map(lambda x: int(x),b))

    Conv ="GAT"# trial.suggest_categorical("conv", ["SAGE","GCN","GAT"])
    
    walk_length = trial.suggest_int("walk length",5,20,step = 5)
    walks_per_node = trial.suggest_int("walks per node", 5,20,step = 5)
    context_size = trial.suggest_int("context size",5,20,step = 5)
    num_negative_samples = walks_per_node
    loss = {"Name":"SAGE" , "walk length":walk_length,"walks per node":walks_per_node,"num negative samples":num_negative_samples,"context size" : context_size,"p":1,"q":1, "loss var": "Random Walks"}

    model = Net(dataset = dataset,mode='unsupervised',conv=Conv,loss_function=loss["loss var"],device=device,hidden_layer=hidden_layer,out_layer =out_layer,num_layers = len(size),dropout = dropout)
    train_loader = NeighborSampler(data.edge_index, node_idx=train_mask, batch_size = 677, sizes=size)
    LossSampler = Sampler(data,device=device,mask=train_mask,loss_info=loss)
    
    model.to(device)
    #Generate the optimizers
#optimizer_name = trial.suggest_categorical("optimizer", ["Adam","RMSprop","SGD"])
    learning_rate=trial.suggest_float("lr",5e-4,1e-2)
    #optimizer = getattr(optim,optimizer_name)(model.parameters(),lr = learning_rate)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay = 1e-5)  
    #classifier = trial.suggest_categorical("classifier", ["logistic regression", "catboost"])
    #classifier = "logistic regression"
    classifier = "catboost"
    #training of the model
    for epoch in range(50):
        loss = train(model,data,optimizer,LossSampler,train_loader,dropout)
        train_acc, test_acc = test(model,data,classifier)
        
    trial.report(test_acc,epoch)
    if trial.should_prune():
        raise optuna.exceptions.TrialsPruned()
    return test_acc
        
    
study = optuna.create_study(direction="maximize",study_name="SAGE loss")
study.optimize(objective,n_trials = 3)
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print(" Number of finid=shed trials: ",len(study.trials))
print(" Number of pruned trials: ", len(pruned_trials))
print(" Number of complete trials: ", len(complete_trials))

print('Best trial:')
trial = study.best_trial
print(" Value: ", trial.value)
print(" Params: ")
for key, value in trial.params.items():
    print(" {}: {}".format(key,value))

In [ ]:
import optuna
from torch_geometric.datasets import Planetoid
dataset =Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())
data = dataset[0]

def objective(trial):
    # Integer parameter
    hidden_layer = 32#trial.suggest_categorical("hidden_layer", [32,64,128,256])
    out_layer = trial.suggest_int("out_layer", 32,128,step = 32,log = False)
    # Floating point parameter
    dropout = trial.suggest_float("dropout", 0.0,0.5,step = 0.1)
   # size_str = trial.suggest_categorical("size", ["[10]","[25]","[5,10]","[10,25]","[5,10,25]"])
   # b = size_str.split("[")[1].split("]")[0].split(',')
    size = [5]#list(map(lambda x: int(x),b))

    Conv ="GAT"# trial.suggest_categorical("conv", ["SAGE","GCN","GAT"])
    
    walk_length = trial.suggest_int("walk length",5,20,step = 5)
    walks_per_node = trial.suggest_int("walks per node", 5,20,step = 5)
    context_size = trial.suggest_int("context size",5,20,step = 5)
    num_negative_samples = walks_per_node
    loss = {"Name":"SAGE" , "walk length":walk_length,"walks per node":walks_per_node,"num negative samples":num_negative_samples,"context size" : context_size,"p":1,"q":1, "loss var": "Random Walks"}

    model = Net(dataset = dataset,mode='unsupervised',conv=Conv,loss_function=loss["loss var"],device=device,hidden_layer=hidden_layer,out_layer =out_layer,num_layers = len(size),dropout = dropout)
    train_loader = NeighborSampler(data.edge_index, node_idx=train_mask, batch_size = 677, sizes=size)
    LossSampler = Sampler(data,device=device,mask=train_mask,loss_info=loss)
    
    model.to(device)
    #Generate the optimizers
#optimizer_name = trial.suggest_categorical("optimizer", ["Adam","RMSprop","SGD"])
    learning_rate=trial.suggest_float("lr",5e-4,1e-2)
    #optimizer = getattr(optim,optimizer_name)(model.parameters(),lr = learning_rate)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay = 1e-5)  
    #classifier = trial.suggest_categorical("classifier", ["logistic regression", "catboost"])
    classifier = "logistic regression"
    #classifier = "catboost"
    #training of the model
    for epoch in range(50):
        loss = train(model,data,optimizer,LossSampler,train_loader,dropout)
        train_acc, test_acc = test(model,data,classifier)
        
    trial.report(test_acc,epoch)
    if trial.should_prune():
        raise optuna.exceptions.TrialsPruned()
    return test_acc
        
    
study = optuna.create_study(direction="maximize",study_name="SAGE loss")
study.optimize(objective,n_trials = 3)
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print(" Number of finid=shed trials: ",len(study.trials))
print(" Number of pruned trials: ", len(pruned_trials))
print(" Number of complete trials: ", len(complete_trials))

print('Best trial:')
trial = study.best_trial
print(" Value: ", trial.value)
print(" Params: ")
for key, value in trial.params.items():
    print(" {}: {}".format(key,value))

In [ ]:
import optuna
from torch_geometric.datasets import Planetoid
dataset =Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())
data = dataset[0]

def objective(trial):
    # Integer parameter
    hidden_layer =32# trial.suggest_categorical("hidden_layer", [32,64,128,256])
    out_layer = trial.suggest_int("out_layer", 32,128,step = 32,log = False)
    # Floating point parameter
    dropout = trial.suggest_float("dropout", 0.0,0.5,step = 0.1)
   # size_str = trial.suggest_categorical("size", ["[10]","[25]","[5,10]","[10,25]","[5,10,25]"])
   # b = size_str.split("[")[1].split("]")[0].split(',')
    size = [10]#list(map(lambda x: int(x),b))

    Conv ="GAT"# trial.suggest_categorical("conv", ["SAGE","GCN","GAT"])
    
    walk_length = trial.suggest_int("walk length",5,20,step = 5)
    walks_per_node = trial.suggest_int("walks per node", 5,20,step = 5)
    context_size = trial.suggest_int("context size",5,20,step = 5)
    num_negative_samples = walks_per_node
    loss = {"Name":"SAGE" , "walk length":walk_length,"walks per node":walks_per_node,"num negative samples":num_negative_samples,"context size" : context_size,"p":1,"q":1, "loss var": "Random Walks"}

    model = Net(dataset = dataset,mode='unsupervised',conv=Conv,loss_function=loss["loss var"],device=device,hidden_layer=hidden_layer,out_layer =out_layer,num_layers = len(size),dropout = dropout)
    train_loader = NeighborSampler(data.edge_index, node_idx=train_mask, batch_size = 677, sizes=size)
    LossSampler = Sampler(data,device=device,mask=train_mask,loss_info=loss)
    
    model.to(device)
    #Generate the optimizers
#optimizer_name = trial.suggest_categorical("optimizer", ["Adam","RMSprop","SGD"])
    learning_rate=trial.suggest_float("lr",5e-4,1e-2)
    #optimizer = getattr(optim,optimizer_name)(model.parameters(),lr = learning_rate)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay = 1e-5)  
    #classifier = trial.suggest_categorical("classifier", ["logistic regression", "catboost"])
    #classifier = "logistic regression"
    classifier = "catboost"
    #training of the model
    for epoch in range(50):
        loss = train(model,data,optimizer,LossSampler,train_loader,dropout)
        train_acc, test_acc = test(model,data,classifier)
        
    trial.report(test_acc,epoch)
    if trial.should_prune():
        raise optuna.exceptions.TrialsPruned()
    return test_acc
        
    
study = optuna.create_study(direction="maximize",study_name="SAGE loss")
study.optimize(objective,n_trials = 3)
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print(" Number of finid=shed trials: ",len(study.trials))
print(" Number of pruned trials: ", len(pruned_trials))
print(" Number of complete trials: ", len(complete_trials))

print('Best trial:')
trial = study.best_trial
print(" Value: ", trial.value)
print(" Params: ")
for key, value in trial.params.items():
    print(" {}: {}".format(key,value))

In [ ]:
import optuna
from torch_geometric.datasets import Planetoid
dataset =Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())
data = dataset[0]

def objective(trial):
    # Integer parameter
    hidden_layer =32# trial.suggest_categorical("hidden_layer", [32,64,128,256])
    out_layer = trial.suggest_int("out_layer", 32,128,step = 32,log = False)
    # Floating point parameter
    dropout = trial.suggest_float("dropout", 0.0,0.5,step = 0.1)
   # size_str = trial.suggest_categorical("size", ["[10]","[25]","[5,10]","[10,25]","[5,10,25]"])
   # b = size_str.split("[")[1].split("]")[0].split(',')
    size = [10]#list(map(lambda x: int(x),b))

    Conv ="GAT"# trial.suggest_categorical("conv", ["SAGE","GCN","GAT"])
    
    walk_length = trial.suggest_int("walk length",5,20,step = 5)
    walks_per_node = trial.suggest_int("walks per node", 5,10,step = 5)
    context_size = trial.suggest_int("context size",5,20,step = 5)
    num_negative_samples = walks_per_node
    loss = {"Name":"SAGE" , "walk length":walk_length,"walks per node":walks_per_node,"num negative samples":num_negative_samples,"context size" : context_size,"p":1,"q":1, "loss var": "Random Walks"}

    model = Net(dataset = dataset,mode='unsupervised',conv=Conv,loss_function=loss["loss var"],device=device,hidden_layer=hidden_layer,out_layer =out_layer,num_layers = len(size),dropout = dropout)
    train_loader = NeighborSampler(data.edge_index, node_idx=train_mask, batch_size = 677, sizes=size)
    LossSampler = Sampler(data,device=device,mask=train_mask,loss_info=loss)
    
    model.to(device)
    #Generate the optimizers
#optimizer_name = trial.suggest_categorical("optimizer", ["Adam","RMSprop","SGD"])
    learning_rate=trial.suggest_float("lr",5e-4,1e-2)
    #optimizer = getattr(optim,optimizer_name)(model.parameters(),lr = learning_rate)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay = 1e-5)  
    #classifier = trial.suggest_categorical("classifier", ["logistic regression", "catboost"])
    classifier = "logistic regression"
    #classifier = "catboost"
    #training of the model
    for epoch in range(50):
        loss = train(model,data,optimizer,LossSampler,train_loader,dropout)
        train_acc, test_acc = test(model,data,classifier)
        
    trial.report(test_acc,epoch)
    if trial.should_prune():
        raise optuna.exceptions.TrialsPruned()
    return test_acc
        
    
study = optuna.create_study(direction="maximize",study_name="SAGE loss")
study.optimize(objective,n_trials = 3)
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print(" Number of finid=shed trials: ",len(study.trials))
print(" Number of pruned trials: ", len(pruned_trials))
print(" Number of complete trials: ", len(complete_trials))

print('Best trial:')
trial = study.best_trial
print(" Value: ", trial.value)
print(" Params: ")
for key, value in trial.params.items():
    print(" {}: {}".format(key,value))

In [ ]:
import optuna
from torch_geometric.datasets import Planetoid
dataset =Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())
data = dataset[0]

def objective(trial):
    # Integer parameter
    hidden_layer = 32#trial.suggest_categorical("hidden_layer", [32,64,128,256])
    out_layer = trial.suggest_int("out_layer", 32,128,step = 32,log = False)
    # Floating point parameter
    dropout = trial.suggest_float("dropout", 0.0,0.5,step = 0.1)
   # size_str = trial.suggest_categorical("size", ["[10]","[25]","[5,10]","[10,25]","[5,10,25]"])
   # b = size_str.split("[")[1].split("]")[0].split(',')
    size = [25]#list(map(lambda x: int(x),b))

    Conv ="GAT"# trial.suggest_categorical("conv", ["SAGE","GCN","GAT"])
    
    walk_length = trial.suggest_int("walk length",5,20,step = 5)
    walks_per_node = trial.suggest_int("walks per node", 5,10,step = 5)
    context_size = trial.suggest_int("context size",5,20,step = 5)
    num_negative_samples = walks_per_node
    loss = {"Name":"SAGE" , "walk length":walk_length,"walks per node":walks_per_node,"num negative samples":num_negative_samples,"context size" : context_size,"p":1,"q":1, "loss var": "Random Walks"}

    model = Net(dataset = dataset,mode='unsupervised',conv=Conv,loss_function=loss["loss var"],device=device,hidden_layer=hidden_layer,out_layer =out_layer,num_layers = len(size),dropout = dropout)
    train_loader = NeighborSampler(data.edge_index, node_idx=train_mask, batch_size = 677, sizes=size)
    LossSampler = Sampler(data,device=device,mask=train_mask,loss_info=loss)
    
    model.to(device)
    #Generate the optimizers
#optimizer_name = trial.suggest_categorical("optimizer", ["Adam","RMSprop","SGD"])
    learning_rate=trial.suggest_float("lr",5e-4,1e-2)
    #optimizer = getattr(optim,optimizer_name)(model.parameters(),lr = learning_rate)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay = 1e-5)  
    #classifier = trial.suggest_categorical("classifier", ["logistic regression", "catboost"])
    #classifier = "logistic regression"
    classifier = "catboost"
    #training of the model
    for epoch in range(50):
        loss = train(model,data,optimizer,LossSampler,train_loader,dropout)
        train_acc, test_acc = test(model,data,classifier)
        
    trial.report(test_acc,epoch)
    if trial.should_prune():
        raise optuna.exceptions.TrialsPruned()
    return test_acc
        
    
study = optuna.create_study(direction="maximize",study_name="SAGE loss")
study.optimize(objective,n_trials = 3)
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print(" Number of finid=shed trials: ",len(study.trials))
print(" Number of pruned trials: ", len(pruned_trials))
print(" Number of complete trials: ", len(complete_trials))

print('Best trial:')
trial = study.best_trial
print(" Value: ", trial.value)
print(" Params: ")
for key, value in trial.params.items():
    print(" {}: {}".format(key,value))

In [ ]:
import optuna
from torch_geometric.datasets import Planetoid
dataset =Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())
data = dataset[0]

def objective(trial):
    # Integer parameter
    hidden_layer = 32#trial.suggest_categorical("hidden_layer", [32,64,128,256])
    out_layer = trial.suggest_int("out_layer", 32,128,step = 32,log = False)
    # Floating point parameter
    dropout = trial.suggest_float("dropout", 0.0,0.5,step = 0.1)
   # size_str = trial.suggest_categorical("size", ["[10]","[25]","[5,10]","[10,25]","[5,10,25]"])
   # b = size_str.split("[")[1].split("]")[0].split(',')
    size = [25]#list(map(lambda x: int(x),b))

    Conv ="GAT"# trial.suggest_categorical("conv", ["SAGE","GCN","GAT"])
    
    walk_length = trial.suggest_int("walk length",5,20,step = 5)
    walks_per_node = trial.suggest_int("walks per node", 5,10,step = 5)
    context_size = trial.suggest_int("context size",5,20,step = 5)
    num_negative_samples = walks_per_node
    loss = {"Name":"SAGE" , "walk length":walk_length,"walks per node":walks_per_node,"num negative samples":num_negative_samples,"context size" : context_size,"p":1,"q":1, "loss var": "Random Walks"}

    model = Net(dataset = dataset,mode='unsupervised',conv=Conv,loss_function=loss["loss var"],device=device,hidden_layer=hidden_layer,out_layer =out_layer,num_layers = len(size),dropout = dropout)
    train_loader = NeighborSampler(data.edge_index, node_idx=train_mask, batch_size = 677, sizes=size)
    LossSampler = Sampler(data,device=device,mask=train_mask,loss_info=loss)
    
    model.to(device)
    #Generate the optimizers
#optimizer_name = trial.suggest_categorical("optimizer", ["Adam","RMSprop","SGD"])
    learning_rate=trial.suggest_float("lr",5e-4,1e-2)
    #optimizer = getattr(optim,optimizer_name)(model.parameters(),lr = learning_rate)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay = 1e-5)  
    #classifier = trial.suggest_categorical("classifier", ["logistic regression", "catboost"])
    classifier = "logistic regression"
    #classifier = "catboost"
    #training of the model
    for epoch in range(50):
        loss = train(model,data,optimizer,LossSampler,train_loader,dropout)
        train_acc, test_acc = test(model,data,classifier)
        
    trial.report(test_acc,epoch)
    if trial.should_prune():
        raise optuna.exceptions.TrialsPruned()
    return test_acc
        
    
study = optuna.create_study(direction="maximize",study_name="SAGE loss")
study.optimize(objective,n_trials = 3)
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print(" Number of finid=shed trials: ",len(study.trials))
print(" Number of pruned trials: ", len(pruned_trials))
print(" Number of complete trials: ", len(complete_trials))

print('Best trial:')
trial = study.best_trial
print(" Value: ", trial.value)
print(" Params: ")
for key, value in trial.params.items():
    print(" {}: {}".format(key,value))